In [87]:
import pandas as pd
import json
import os


In [88]:
json_files = os.listdir('StreamingHistory/')
combined_json = []

for file in json_files:
    with open(f"StreamingHistory/{file}", 'r') as f:
        data = json.load(f)
        combined_json.extend(data)

df = pd.DataFrame(combined_json)
df.to_csv('all_stream_history.csv', index=False)


In [89]:
df.head(1)


,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2012-04-26T04:00:05Z,1227567202,Windows 7 (Unknown Ed) SP0 [x86 0],207476,US,131.247.32.107,None,Good Girl,Carrie Underwood,Good Girl,...,None,None,None,popup,endplay,False,False,False,0,False


In [90]:
df = df.rename(columns={
    'ts': 'timestamp',
    'master_metadata_track_name': 'track_name',
    'master_metadata_album_artist_name': 'artist_name',
    'master_metadata_album_album_name': 'album_name',
    'spotify_track_uri': 'track_uri'
})


In [91]:
columns_to_drop = ["username", "platform", "ms_played", "conn_country", "ip_addr_decrypted", "user_agent_decrypted", "episode_name", "episode_show_name",
"spotify_episode_uri", "reason_start", "reason_end", "shuffle", "skipped", "offline", "offline_timestamp", "incognito_mode"]


In [92]:
df.drop(columns=columns_to_drop, inplace=True)


In [93]:
df.to_csv('all_stream_history_2.csv', index=False)


In [94]:
data = pd.read_csv('all_stream_history_2.csv')


In [99]:
data.sample(20)


,timestamp,track_name,artist_name,album_name,track_uri
8389,2013-11-25T19:49:51Z,Twisted - Single Version; 2007 Remaster,Keith Sweat,The Best of Keith Sweat: Make You Sweat,spotify:track:4h067wpWDHwQWM4a6CbqSV
31634,2022-11-02T04:07:52Z,Question...?,Taylor Swift,Midnights,spotify:track:0heeNYlwOGuUSe7TgUD27B
19304,2018-06-30T03:27:35Z,End Game,Taylor Swift,reputation,spotify:track:2x0WlnmfG39ZuDmstl9xfX
16010,2014-01-25T04:11:09Z,One,Swedish House Mafia,One (Your Name),spotify:track:4xr1Azj8mCbz6u8CITf8ef
13482,2014-01-10T23:23:21Z,Intro,The xx,xx,spotify:track:0DAsxISzun85PbsqAfIzeC
42788,2017-07-21T13:54:13Z,Relationship (feat. Future),Young Thug,Beautiful Thugger Girls,spotify:track:25oOaleife6E2MIKmFkPvg
17103,2018-05-20T19:08:57Z,Same Old Love,Selena Gomez,Revival,spotify:track:1BZG99C7Co1r6QUC3zaS59
23867,2021-12-14T01:50:00Z,Slow Down (feat. Lucky Daye),VanJess,Slow Down (feat. Lucky Daye),spotify:track:6EsOT4Yq4BavQSJ4sEe9dw
13237,2014-01-09T00:26:07Z,3 - Manhattan Clique Remix Club,Britney Spears,"B In The Mix, The Remixes Vol 2",spotify:track:0S2yXbHxgNr8nrMTJOk7gK
43357,2017-07-24T03:49:55Z,This Time,Carrie Underwood,Play On,spotify:track:3onIXgrZ2k6F0Luj81su9v


In [133]:
data['track_name'].value_counts().head(50)


track_name
Lights - Single Version                                 432
Shake It Off                                            322
Deeper Sleep                                            185
The Early Morning Rain                                  147
Kill Bill                                               136
Roar                                                    131
Let Me Go (with Alesso, Florida Georgia Line & watt)    123
Fast Car                                                118
Still into You                                          112
It Ain’t Me (with Selena Gomez)                         110
Snooze                                                  109
The Cure                                                108
Ain't It Fun                                            104
Ghost                                                   104
Chicago                                                 101
I Don’t Wanna Live Forever (Fifty Shades Darker)         99
100 Letters                  

In [134]:
unique_songs = data.drop_duplicates(subset=['track_name', 'artist_name'])
unique_songs = unique_songs.dropna()


In [158]:
unique_songs.shape


(22471, 5)

In [136]:
tracks_to_remove = ['Deeper Sleep', 'The Early Morning Rain', 'Chicago', 'White Noise For Baby Sleep']
unique_songs = unique_songs[~data['track_name'].isin(tracks_to_remove)]


/var/folders/t6/bfyvsvq54r1f1_xd_gjqncpr0000gn/T/ipykernel_76176/2642932415.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unique_songs = unique_songs[~data['track_name'].isin(tracks_to_remove)]


In [160]:
unique_songs['artist_name'].value_counts().head(10)


artist_name
Taylor Swift                  325
Drake                         159
Nicki Minaj                   126
Maroon 5                      113
White Noise For Baby Sleep    112
Demi Lovato                   111
Beyoncé                       108
Britney Spears                104
Carrie Underwood              102
Miley Cyrus                   100
Name: count, dtype: int64

In [161]:
usher_tracks = unique_songs[unique_songs['artist_name'] == 'Billie Eilish']


In [162]:
remix_tracks = unique_songs[unique_songs['track_name'].str.contains('remix', case=False, na=False)]


In [154]:
remix_tracks.head()


,timestamp,track_name,artist_name,album_name,track_uri
500,2012-06-24T19:50:20Z,Where Have You Been - Papercha$er Remix,Rihanna,Where Have You Been,spotify:track:19f9jLCf7yDO9fsmuKGRMo
572,2012-06-25T01:48:48Z,Raise Your Weapon - Madeon Extended Remix,deadmau5,Toolroom Records Ibiza 2011 Vol. 2,spotify:track:3FvrHJH1p0bQ7eeXlvjVfc
573,2012-06-25T01:53:26Z,Cinema - Skrillex Remix,Benny Benassi,Cinema,spotify:track:39tf7BmdD6n18iWYkgRHIP
574,2012-06-25T04:46:08Z,Midnight City - Eric Prydz Private Remix,M83,Midnight City,spotify:track:6pNBH4KKQYVRtgXNrINq1w
700,2012-07-02T01:24:15Z,Almost Lover - Remix,A Fine Frenzy,One Cell In The Sea,spotify:track:3oOeYbQd3fH8CsRfhF4BuU


In [163]:
unique_songs.shape


(22471, 5)

In [164]:
unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


/var/folders/t6/bfyvsvq54r1f1_xd_gjqncpr0000gn/T/ipykernel_76176/1976657215.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


In [165]:
unique_songs.shape


(21638, 5)

In [167]:
unique_songs.sample(25)


,timestamp,track_name,artist_name,album_name,track_uri
22650,2021-10-13T21:51:04Z,Mood (feat. iann dior),24kGoldn,El Dorado,spotify:track:4jPy3l0RUwlUI9T5XHBW2m
11984,2013-12-24T11:51:00Z,All over You,The Spill Canvas,"No Really, I'm Fine",spotify:track:4b9jRNwMUvABxTD1amyfA6
68688,2023-06-03T04:15:51Z,Vibration,Fireboy DML,"Laughter, Tears & Goosebumps",spotify:track:7u0z6FcJDqkZwdpqAqUzmO
58187,2023-01-01T19:39:43Z,Trigger,Hayley Williams,FLOWERS for VASES / descansos,spotify:track:0GDObY1nXkQCYFNYpW2pAv
67705,2023-06-02T21:41:54Z,Pink Lemonade,Johnny Stimson,Flower,spotify:track:55fTqSc1aVBtdHcGmP8TSj
16182,2014-01-27T01:39:05Z,Begin Again,Jeff Hendrick,Begin Again,spotify:track:7sDalo11657flWPeEagRU9
59043,2023-01-16T15:03:07Z,Fake Plastic Trees,Kelly Clarkson,Kellyoke,spotify:track:6XoNven2kQcDudtwsxOCfj
38763,2024-01-07T22:56:01Z,Drops of Jupiter (Tell Me),Train,Drops Of Jupiter,spotify:track:2hKdd3qO7cWr2Jo0Bcs0MA
79085,2014-04-08T02:36:24Z,Mmm Yeah,Austin Mahone,Mmm Yeah,spotify:track:20zQs2LhTCzA2PfIPMxuWu
2384,2013-05-16T01:54:53Z,Our Song,Taylor Swift,Taylor Swift,spotify:track:15DeqWWQB4dcEWzJg15VrN


In [ ]:
# need to filter by how long i listen. Anything under a 2 minutes, I probably didn't like,
